In [25]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.graphics.regressionplots import plot_partregress_grid

from statsmodels.tools.eval_measures import mse, rmse
from statsmodels.iolib.smpickle import load_pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [26]:
# loading project data
data = pd.read_csv('us_bank_wages/us_bank_wages.txt', delimiter='\t')
data.to_csv('us_bank_wages/us_bank_wages_data.csv')

In [27]:
# checking for NaN 
data.isnull().sum()
data.isnull().values.any()

list(data.columns)
data.drop('Unnamed: 0', axis=1, inplace=True)

# de-skewing - explaination: the SALERY colmuns are right skewed and to compensate for that I use log() 
data.eval('LSALARY = log(SALARY)', inplace = True)
data.eval('LSALBEGIN = log(SALBEGIN)', inplace = True)

In [28]:
# make work copy of data

work = data.copy();

# inside the work copy the coded values are converted to categorical values - for better readability 

def map_educ(x):
    if x == 8: return 'primary'
    elif x == 12: return 'secondary'
    elif x > 12: return 'university'
work['EDUC'] = work.EDUC.apply(map_educ).astype('category')

map_dict = {0: 'female', 1:'male'}
work['GENDER'] = work['GENDER'].map(map_dict)
map_dict = {0: 'non-minorities', 1:'minorities'}
work['MINORITY'] = work['MINORITY'].map(map_dict)
map_dict = {1: 'administrative', 2:'custodial', 3:'management'}
work['JOBCAT'] = work['JOBCAT'].map(map_dict)

In [29]:
# spliting data into train and test data

train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True) # random_state = seed !!

In [30]:
# define function for brute force computation of all models
def find_best_model(var,log=False):
    r = {}
    for t in var:
        if log:
            f = 'LSALARY ~ ' + ' + '.join(t)
        else:
            f = 'SALARY ~ ' + ' + '.join(t)
        m = smf.ols(formula=f, data=train).fit()
        r[m.rsquared_adj] = f # m.rsquared, m.f_pvalue, np.sqrt(m.mse_resid)
    return r

In [31]:
# generate column combinations - to check various models

var = []
columns = ['SALBEGIN','LSALBEGIN','GENDER','C(MINORITY)','C(JOBCAT)','C(EDUC)']
for i in range(len(columns)):
    # remove first
    cols = columns[i+1:]
    # add first to end
    cols += columns[:i+1]
    #print(cols)
    
    for i in range(len(cols)):
        if not [cols[i]] in var:
            var.append([cols[i]])
            
        c = cols.copy()
        c.remove(cols[i])
        while len(c):
            if not sorted([x for x in c]) in var:
                var.append(sorted([x for x in c]))
            x = c.pop() # x is only used to have less print-output in JL

In [43]:
# run brute force model computation
model_fit_results = find_best_model(var,False)

# list model fittings - top 10
model_fit_result_list = sorted(model_fit_results.keys())[-6:]
for fit in model_fit_result_list:
    #print('rsquared_adj:', fit, '\t<-', model_fit_results[fit])
    pass
    
model_fit_use = model_fit_results[model_fit_result_list[-4]]

model_fit_use

'SALARY ~ C(EDUC) + C(JOBCAT) + SALBEGIN'

In [42]:
train_model = smf.ols(formula=model_fit_use, data=train)
train_model_fit = train_model.fit()

#train_model_fit.params

In [44]:
train_model_predict = train_model_fit.predict(train);
train_model_actual = train['SALARY'].astype(float);

print("Mean Absolute Error (MAE)        : {}".format(
    mean_absolute_error(train_model_actual, train_model_predict)))
print("Mean Squared Error (MSE)         : {}".format(
    mse(train_model_actual, train_model_predict)))
print("Root Mean Squared Error (RMSE)   : {}".format(
    rmse(train_model_actual, train_model_predict)))
print("Mean Absolute Perc. Error (MAPE) : {}".format(
    np.mean(np.abs((train_model_actual - train_model_predict) / train_model_actual)) * 100))

Mean Absolute Error (MAE)        : 4762.818258860782
Mean Squared Error (MSE)         : 53924465.39180361
Root Mean Squared Error (RMSE)   : 7343.3279507185025
Mean Absolute Perc. Error (MAPE) : 13.143136437260466


In [45]:
def save_model(model_fit, file='default.pickle'):
    model_fit.save(file)

In [46]:
def read_model(file='default.csv'):
    model_fit = load_pickle(file)
    return model_fit

In [47]:
def save_csv(data: pd.DataFrame, file='default.csv'):
    data.to_csv(file)